In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#getting the dataset into the notebook
df=pd.read_csv('/kaggle/input/pokemon/Pokemon.csv')

In [ ]:
#obviously gotta do this at the start of every kaggle code
df.head()

In [ ]:
#Replacing the pokemon that don't have a second type to say that 
df['Type 2'].fillna("No Type2", inplace = True)
df[df ['Type 2'].isna()]

In [ ]:
#Getting rid of the megas and primals,as they had the same Pokedex numbers and we didin't want that to mess with anything in the future 
nm = df.loc[(df['Name'].str.contains('Mega ')==False) & (df['Name'].str.contains('Primal')==False)]
nm.head(20)

In [ ]:
#This is where we get the pokemon we are going to use
#This will also let you re-enter a pokemon in case you messed up the spelling or typed some random stuff
def poke_get(question):
    chosen_poke1 = input(question)
    if chosen_poke1.title() in nm['Name'].values:
        poke_info1 = nm.loc[nm['Name'] == chosen_poke1.title()]
        poke_num1 = poke_info1.index[0]
        return poke_num1, chosen_poke1.title()
    else: 
        return poke_get(question)
poke_num1, chosen_poke1 = poke_get('What is your first Pokemon? ')
poke_num2, chosen_poke2 = poke_get('What is your second Pokemon? ')

In [ ]:
#This is getting the attack type that each poke will use
#Based on which is higher, attack or special attack
def attack(poke_num):
    pokemon_attack = nm.loc[poke_num, 'Attack']
    pokemon_attack
    
    pokemon_spattack = nm.loc[poke_num, 'Sp. Atk']
    pokemon_spattack
    if pokemon_attack > pokemon_spattack:
        return pokemon_attack, 'Attack'
    else:
        return pokemon_spattack, 'Sp. Atk'

In [ ]:
#checking attack type for our first Pokemon
poke_attack_first = attack(poke_num1)
poke_attack_first

In [ ]:
#checking attack type for our second Pokemon
poke_attack_second = attack(poke_num2)
poke_attack_second

In [ ]:
#This is where we determine which Pokemon will attack first based on the speed stat
def speed(poke_num1, poke_num2, chosen_poke1,chosen_poke2):
    pokemon_speed1 = nm.loc[poke_num1, 'Speed']
    pokemon_speed1
    
    pokemon_speed2 = nm.loc[poke_num2, 'Speed']
    pokemon_speed2

    if pokemon_speed1 > pokemon_speed2:
        return poke_num1, chosen_poke1
    else:
        return poke_num2, chosen_poke2

In [ ]:
#This is where we get the slower Pokemon to save for later
#In case the first pokmemon's attack isn't greater than the other Pokemon's defenses
def slower(poke_num1, poke_num2, chosen_poke1,chosen_poke2):
    pokemon_speed1 = nm.loc[poke_num1, 'Speed']
    pokemon_speed1
    
    pokemon_speed2 = nm.loc[poke_num2, 'Speed']
    pokemon_speed2

    if pokemon_speed1 < pokemon_speed2:
        return poke_num1, chosen_poke1
    else:
        return poke_num2, chosen_poke2

In [ ]:
#Getting the attacks again, as a precaution to make sure everything works correctly
faster_poke = speed(poke_num1, poke_num2, chosen_poke1,chosen_poke2)
faster_poke_attack = attack(faster_poke[0])

slower_poke = slower(poke_num1, poke_num2, chosen_poke1,chosen_poke2)
slower_poke_attack = attack(slower_poke[0])

In [ ]:
#This is for getting the defense type for the defending Pokemon
#Which we will use to compare the the attack to
#Sp. Atk -> Sp. Def and Attack -> Defense
def defense(slower_poke, faster_poke_attack):
    pokemon_defense = nm.loc[slower_poke[0], 'Defense']
    pokemon_defense
    
    pokemon_spdefense = nm.loc[slower_poke[0], 'Sp. Def']
    pokemon_spdefense
    #I know this doesn't work but it gets the idea across
    if faster_poke_attack == 'Sp. Atk':
        return pokemon_spdefense, 'Sp. Def'
    else:
        return pokemon_defense, 'Defense'
first_defense = defense(slower_poke, faster_poke_attack)
first_defense

In [ ]:
#This is in case the first hit doesn't KO the Pokemon that defended first
#The same as above, but getting the faster Pokemon's defenses
def faster_defense(faster_poke, slower_poke_attack):
    pokemon_defense = nm.loc[faster_poke[0], 'Defense']
    pokemon_defense
    
    pokemon_spdefense = nm.loc[faster_poke[0], 'Sp. Def']
    pokemon_spdefense
    #I know this doesn't work but it gets the idea across
    if slower_poke_attack == 'Sp. Atk':
        return pokemon_spdefense, 'Sp. Def'
    else:
        return pokemon_defense, 'Defense'
second_defense = faster_defense(faster_poke, slower_poke_attack)
second_defense

In [ ]:
#This will only run in VERY special cases where neither pokemon can knock out the other in one hit
#This is the tie breaker meant to symbolise a more drawn out fight
def hp_count(chosen_poke1, chosen_poke2, poke_num1, poke_num2):
    hp_poke1 = nm.loc[poke_num1, 'HP']
    hp_poke2 = nm.loc[poke_num2, 'HP']
    if hp_poke1 > hp_poke2:
        return chosen_poke1
    else: 
        return chosen_poke2

In [ ]:
#This function chooses the winner based on comparing attacks and defenses
#It reads pretty straight forward, but the faster Pokemon attacks and compares the defense of the slower Pokemon
#If the attack is higher, the Faster Pokemon wins
#If the attack is lower it compares the attack of the slower Pokemon and defense of the faster Pokemon
#If that attack is higher, the Slower Pokemon wins
#If neither are higher it compares the health, to show a longer fight, and chooses the winner that way
def winner(faster_poke_attack, first_defense, faster_poke, slower_poke, second_defense, slower_poke_attack, chosen_poke1, chosen_poke2, poke_num1, poke_num2):
    first_defense = first_defense[0]
    faster_poke_attack = faster_poke_attack[0]
    second_defense = second_defense[0]
    slower_poke_attack = slower_poke_attack[0]
    if faster_poke_attack > first_defense:
        return 'This is the winner!', faster_poke[1]
    else:
        if slower_poke_attack > second_defense:
            return 'This is the winner', slower_poke[1]
        else:
            tie_breaker = hp_count(chosen_poke1, chosen_poke2, poke_num1, poke_num2)
            return 'This is the winner', tie_breaker
            
        
winner(faster_poke_attack, first_defense, faster_poke, slower_poke, second_defense, slower_poke_attack, chosen_poke1, chosen_poke2, poke_num1, poke_num2)